In [1]:
from ultralytics import YOLO
from pathlib import Path

root = Path("/mnt/d/Defect-Detection")
out_dir = root / "models" / "yolov8-det" / "latest"
out_dir.mkdir(parents=True, exist_ok=True)

print("Exporting YOLOv8n to ONNX (imgsz=960, opset=12)…")
m = YOLO("yolov8n.pt")        # pretrained COCO
onnx_path = m.export(format="onnx", imgsz=960, opset=12, dynamic=False)
# ultralytics trả về đường dẫn file vừa xuất (vd 'yolov8n.onnx')
Path(onnx_path).rename(out_dir / "model.onnx")

# COCO 80 lớp
coco = [
 'person','bicycle','car','motorbike','aeroplane','bus','train','truck','boat','traffic light',
 'fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow',
 'elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee',
 'skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard','tennis racket',
 'bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich',
 'orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','sofa',
 'pottedplant','bed','diningtable','toilet','tvmonitor','laptop','mouse','remote','keyboard','cell phone',
 'microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear',
 'hair drier','toothbrush'
]
(out_dir / "labels.txt").write_text("\n".join(coco), encoding="utf-8")

print("✔ Wrote:", out_dir / "model.onnx")
print("✔ Wrote:", out_dir / "labels.txt")

Exporting YOLOv8n to ONNX (imgsz=960, opset=12)…


100%|██████████| 6.25M/6.25M [00:46<00:00, 141kB/s] 


Ultralytics 8.3.0 🚀 Python-3.11.8 torch-2.5.1+cu121 CPU (AMD Ryzen 7 7735HS with Radeon Graphics)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 960, 960) BCHW and output shape(s) (1, 84, 18900) (6.2 MB)
requirements: Ultralytics requirement ['onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 462.8 kB/s  0:00:44m0:00:0100:02

requirements: AutoUpdate success ✅ 71.9s, installed 1 package: ['onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 12...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 78.7s, saved as 'yolov8n.onnx' (12.4 MB)

Export complete (79.4s)
Results saved to /mnt/d/Defect-Detection
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=960  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=960 

In [3]:
!docker exec -i infra-clickhouse-1 clickhouse-client --multiquery <<'SQL'
/* LATENCY 5 PHÚT (p50/p95) */

CREATE TABLE IF NOT EXISTS aoi.latency_5m
(
    `t_5m`         DateTime,
    `product_code` LowCardinality(String),
    `station_id`   LowCardinality(String),
    `q_state`      AggregateFunction(quantilesTiming, 0.5, 0.95)
)
ENGINE = AggregatingMergeTree
PARTITION BY toYYYYMM(t_5m)
ORDER BY (t_5m, product_code, station_id)
SETTINGS index_granularity = 8192;

CREATE MATERIALIZED VIEW IF NOT EXISTS aoi.mv_latency_5m
TO aoi.latency_5m
AS
SELECT
    toStartOfFiveMinute(toDateTime(ts_ms / 1000)) AS t_5m,
    product_code,
    station_id,
    quantilesTimingState(0.5, 0.95)(latency_ms) AS q_state
FROM aoi.aoi_inspections
GROUP BY t_5m, product_code, station_id;

CREATE OR REPLACE VIEW aoi.v_latency_5m AS
SELECT
    t_5m,
    product_code,
    station_id,
    quantilesTimingMerge(0.5, 0.95)(q_state)[1] AS p50_ms,
    quantilesTimingMerge(0.5, 0.95)(q_state)[2] AS p95_ms
FROM aoi.latency_5m;
SQL

SyntaxError: invalid syntax (3576524003.py, line 2)